In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 1.8 MB/s eta 0:00:00


In [3]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import confusion_matrix
from wordcloud import WordCloud
import nltk
import re
import string
import subprocess
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import torch
from torch.nn.functional import softmax
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset
from transformers import DistilBertTokenizer
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm.auto import tqdm
from transformers import DistilBertForSequenceClassification
import torch

In [4]:
!pip install -U nltk

In [5]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [6]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
import re, string, unicodedata
from string import punctuation
from nltk.corpus import wordnet

In [7]:
datasets = '/content/drive/MyDrive/datasets/'

In [8]:
books_rating_path = datasets + 'Books_rating.csv'
books_data_path = datasets + 'books_data.csv'

book_rating = pd.read_csv(books_rating_path)
book_data = pd.read_csv(books_data_path)

**Data Pre-Processing**

In [9]:
books = pd.merge(book_rating,book_data, on = 'Title')
books.shape

df = books[['Title','review/score','review/text','authors','categories','ratingsCount']]

df.drop_duplicates(inplace = True)
df.shape

df.isna().sum()

df.dropna(inplace = True)
df.isna().sum()

df.info()

df.shape

<ipython-input-9-1ed62d600c89>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace = True)
<ipython-input-9-1ed62d600c89>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace = True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1325891 entries, 47 to 2999999
Data columns (total 6 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   Title         1325891 non-null  object 
 1   review/score  1325891 non-null  float64
 2   review/text   1325891 non-null  object 
 3   authors       1325891 non-null  object 
 4   categories    1325891 non-null  object 
 5   ratingsCount  1325891 non-null  float64
dtypes: float64(2), object(4)
memory usage: 70.8+ MB


(1325891, 6)

In [10]:
data = df.sample(20000)
data.shape

(20000, 6)

In [11]:
columns_to_extract = ['authors', 'categories']

extraction_pattern = r'\'(.*)\''

for column in columns_to_extract:
    data[column] = data[column].str.extract(extraction_pattern)

data['word_count'] = data['review/text'].str.split().apply(len)

**Sentiment Analysis**

In [12]:
analyzer = SentimentIntensityAnalyzer()

In [13]:
data['clean_reviews'] = data['review/text'].str.lower()
data['score'] = data['clean_reviews'].apply(lambda review: analyzer.polarity_scores(review))

In [14]:
data['compound'] = data['review/text'].apply(lambda review: analyzer.polarity_scores(review.lower())['compound'])


In [15]:
data['Sentiment'] = data['compound'].apply(lambda x: 'positive' if x >= 0.05 else 'negative' if x < -0.05 else 'neutral')

**DataFrame comparing Reviews and Sentiments**

In [16]:
reviews_df = data[['clean_reviews', 'Sentiment']]


In [17]:
reviews_df.head(50)

,clean_reviews,Sentiment
2999456,kale's book is for sap professionals who are i...,positive
1173232,i like the book because we got to chsnge the n...,positive
1761368,the atory is written through scarlets eyes . i...,negative
208327,"the princess bride, s. morgenstern's classic t...",positive
2334928,show of evil is a thrilling sequel to primal f...,positive
388956,i just finished this book for the first time e...,positive
1280884,there's something special about lee child's ja...,positive
1982501,yeech! i don't know why i bothered. i love reg...,negative
440787,"excellent and quick read, notwithstanding the ...",positive
2037332,this small book is a declaration of war on chr...,negative


In [18]:
x = reviews_df.drop(['Sentiment'], axis=1)
y = reviews_df['Sentiment']

**Text Normalization and Preprocessing**

In [19]:
stemmer = nltk.SnowballStemmer("english")
stopword=set(stopwords.words('english'))

In [20]:
stopword = {word.lower() for word in stopword}

url_pattern = re.compile(r'http\S+')
special_chars_pattern = re.compile(r'[^a-zA-Z0-9\s]')

lemmatizer = WordNetLemmatizer()

def clean(text):
    text = url_pattern.sub('', text)

    text = special_chars_pattern.sub('', text)

    words = text.split()
    lemmatized_and_filtered = [
        lemmatizer.lemmatize(word.lower()) for word in words
        if word.lower() not in stopword and word.isalpha()
    ]

    final_text = " ".join(lemmatized_and_filtered)

    return final_text


In [21]:
x["clean_reviews"] = x["clean_reviews"].apply(clean)

In [22]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25,random_state=0)

In [23]:
x_train.head()

,clean_reviews
1222143,good book whet one appetite financial text tru...
262384,hoping find grill book instead book pretty muc...
2310924,settled large volume transatlantic flight thor...
378669,book reviewuglies scott westerfeld p know fift...
811578,enjoyed book read two previous book series loo...


In [24]:
y_train.head()

1222143    positive
262384     positive
2310924    positive
378669     positive
811578     positive
Name: Sentiment, dtype: object

In [25]:
tfidf_vectorizer = TfidfVectorizer()

x_train_tfidf = tfidf_vectorizer.fit_transform(x_train['clean_reviews'])

x_test_tfidf = tfidf_vectorizer.transform(x_test['clean_reviews'])

x_train_tfidf_df = pd.DataFrame(x_train_tfidf.toarray())
x_test_tfidf_df =  pd.DataFrame(x_test_tfidf.toarray())

In [26]:
oversample = SMOTE()

In [27]:
x_train_bal, y_train_bal = oversample.fit_resample(x_train_tfidf, y_train)

In [28]:
x_test_bal, y_test_bal = oversample.fit_resample(x_test_tfidf, y_test)

In [35]:

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize data
train_encodings = tokenizer(list(x_train['clean_reviews']), truncation=True, padding=True, max_length=256, return_tensors="pt")
test_encodings = tokenizer(list(x_test['clean_reviews']), truncation=True, padding=True, max_length=256, return_tensors="pt")


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [36]:

class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [37]:


model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(set(labels)))


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

train_dataset = SentimentDataset(train_encodings, y_train_encoded)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [39]:
from tqdm.auto import tqdm

epochs = 3

for epoch in range(epochs):
    model.train()
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optim.step()

        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())


  0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/938 [00:00<?, ?it/s]

In [44]:
test_dataset = SentimentDataset(test_encodings, y_test_encoded)  # Assuming y_test_encoded is already prepared
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [45]:

model.eval()

predictions = []
true_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        probs = softmax(logits, dim=-1)
        pred_labels = torch.argmax(probs, dim=-1)

        predictions.extend(pred_labels.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')


Accuracy: 0.8982
Precision: 0.8914
Recall: 0.8982
F1 Score: 0.8931


In [49]:
torch.save(model.state_dict(), "distilbert_sentiment_analysis.pth")
tokenizer.save_pretrained("distilbert_tokenizer")


('distilbert_tokenizer/tokenizer_config.json',
 'distilbert_tokenizer/special_tokens_map.json',
 'distilbert_tokenizer/vocab.txt',
 'distilbert_tokenizer/added_tokens.json')

In [58]:
sentence = "This is a fantastic movie!"
inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=256)

model.to('cpu')
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
probabilities = torch.nn.functional.softmax(logits, dim=-1)
predicted_class = torch.argmax(probabilities, dim=-1)

label_map = {0: 'negative', 1: 'neutral', 2: 'positive'}
predicted_label = label_map[predicted_class.item()]

print(f"Predicted sentiment: {predicted_label}")


Predicted sentiment: positive


In [59]:

data = {
    "sentence id": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    "text": [
        "I wouldn't be caught dead watching the NFL if it weren't for Taylor Swift.",
        "Chris O'Donnell stated that while filming for this movie, he felt like he was in a Toys ''R'' Us commercial.",
        "The whole game was a rollercoaster ride, but Los Angeles Lakers ultimately persevered and won!",
        "Zendaya slayed in Dune 2, as she does in all her movies.",
        "While my favorite player was playing this match and started off strongggg, it went downhill after Messi's injyry midgame.",
        "My uncle's brother's neighbor's cat's veterinarian David reads the communist manifesto in his spare time.",
        "He said that The Great Gatsby is the best novell ever, and I was about to throw hands.",
        "I could not look away from this train wrck of a movie, on February 14th of all days.",
        "The film Everything Everywhere All At Once follows Evelyn Wang, a woman drowning under the stress of her family's failing laundromat.",
        "I just finished reading pride and prejudice which had me HOOOKED from the beginning."
    ],
    "sentiment": ["negative", "neutral", "positive", "positive", "negative", "neutral", "negative", "negative", "neutral", "positive"],
    "topic": ["sports", "movie", "sports", "movie", "sports", "book", "book", "movie", "movie", "book"]
}

df = pd.DataFrame(data)


In [61]:



model.eval()

df = pd.DataFrame(data)

tokenizer = DistilBertTokenizer.from_pretrained("distilbert_tokenizer")
encoded_input = tokenizer(df['text'].tolist(), padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    outputs = model(**encoded_input)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1).numpy()

label_map = {0: 'negative', 1: 'neutral', 2: 'positive'}
decoded_predictions = [label_map[pred] for pred in predictions]

df['predicted_sentiment'] = decoded_predictions

print(df[['text', 'sentiment', 'predicted_sentiment']])

kappa_score = cohen_kappa_score(df['sentiment'], df['predicted_sentiment'])
print(f"Kappa Score: {kappa_score}")


                                                text sentiment  \
0  I wouldn't be caught dead watching the NFL if ...  negative   
1  Chris O'Donnell stated that while filming for ...   neutral   
2  The whole game was a rollercoaster ride, but L...  positive   
3  Zendaya slayed in Dune 2, as she does in all h...  positive   
4  While my favorite player was playing this matc...  negative   
5  My uncle's brother's neighbor's cat's veterina...   neutral   
6  He said that The Great Gatsby is the best nove...  negative   
7  I could not look away from this train wrck of ...  negative   
8  The film Everything Everywhere All At Once fol...   neutral   
9  I just finished reading pride and prejudice wh...  positive   

  predicted_sentiment  
0            negative  
1            positive  
2             neutral  
3             neutral  
4            positive  
5             neutral  
6            positive  
7             neutral  
8            negative  
9            negative  
Cohen's K